In [ ]:
# Paths - Update locally!
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [ ]:
import mne
import numpy as np
import pandas as pd
import os
import re
import csv
from tqdm.notebook import tqdm
import warnings

from img_pipe import img_pipe

from matplotlib import pyplot as plt
from matplotlib import rcParams as rc
import matplotlib.patheffects as PathEffects
rc['pdf.fonttype'] = 42
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
subjs = [s for s in os.listdir(
    os.path.join(git_path,"preprocessing","events","csv")) if "TCH" in s or "S0" in s]
exclude = ["TCH8"]
no_imaging = ["S0010"]
subjs = [s for s in subjs if s not in exclude]

blocks = {
    s: [
        b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"analysis","events","csv",s)) if f"{s}_B" in b and os.path.isfile(os.path.join(
            git_path,"analysis","events","csv",s,b,f"{b}_spkr_sn_all.txt"
        ))
    ] for s in subjs
}

# these are the subjects that don't have inconsistent playback trials and
# are therefore excluded from analysis in this notebook
no_sh = ['S0023','TCH06'] 

### Epoch data

In [ ]:
click_tmin, click_tmax = 0.4, 0.6
task_tmin, task_tmax = 0.05, 0.55
baseline = None
epochs, ch_names = dict(), dict()
for s in tqdm(subjs):
    epochs[s] = dict()
    click_epochs, el_epochs, sh_epochs = [], [], []
    for b in blocks[s]:
        blockid = "_".join([s,b])
        raw = mne.io.read_raw_fif(os.path.join(data_path,f"{s}_complete",s,blockid,
            "HilbAA_70to150_8band","ecog_hilbAA70to150.fif"),preload=True, verbose=False)
        fs = raw.info['sfreq']
        # Click events
        click_eventfile = os.path.join(git_path,"preprocessing","events","csv",s,blockid,
                                       f"{blockid}_click_eve.txt")
        with open(click_eventfile,'r') as f:
            r = csv.reader(f, delimiter='\t')
            if s not in ["S0026","TCH14"]:
                events = np.array([[np.ceil(float(row[0])*fs).astype(int),
                                    np.ceil(float(row[2])*fs).astype(int),int(float(row[4]))] for row in r])
            else:
                events = np.loadtxt(click_eventfile).astype(int)
                events[:,0] = np.array(events[:,0] * fs).astype(int)
                events[:,1] = np.array(events[:,1] * fs).astype(int)
        click_epochs.append(mne.Epochs(raw, events, tmin=click_tmin, tmax=click_tmax, baseline=baseline,
            preload=True, verbose=False))
        # Consistent events
        el_eventfile = os.path.join(git_path,"preprocessing","events","csv",s,blockid,
                                      f"{blockid}_spkr_sn_el.txt")
        with open(el_eventfile,'r') as f:
            r = csv.reader(f, delimiter='\t')
            events = np.array([[np.ceil(float(row[0])*fs).astype(int),
                                np.ceil(float(row[1])*fs).astype(int),int(row[2])] for row in r])
        el_epochs.append(mne.Epochs(raw, events, tmin=task_tmin, tmax=task_tmax, baseline=baseline,
                                      preload=True, verbose=False))
        # Mic events
        sh_eventfile = os.path.join(git_path,"preprocessing","events","csv",s,blockid,
                                     f"{blockid}_spkr_sn_sh.txt")
        with open(sh_eventfile,'r') as f:
            r = csv.reader(f, delimiter='\t')
            events = np.array([[np.ceil(float(row[0])*fs).astype(int),
                                np.ceil(float(row[1])*fs).astype(int),int(row[2])] for row in r])
        sh_epochs.append(mne.Epochs(raw, events, tmin=task_tmin, tmax=task_tmax, baseline=baseline,
                                     preload=True, verbose=False))
    epochs[s]['click'] = mne.concatenate_epochs(click_epochs)
    epochs[s]['el'] = mne.concatenate_epochs(el_epochs)
    epochs[s]['sh'] = mne.concatenate_epochs(sh_epochs)
    ch_names[s] = raw.info['ch_names']

### Run the bootstrap

In [ ]:
nboots = 1000; pvals = dict()
pvals['el'] = {s:{ch:0 for ch in ch_names[s]} for s in subjs if s not in no_sh}
pvals['sh'] = {s:{ch:0 for ch in ch_names[s]} for s in subjs if s not in no_sh}
for s in tqdm([ss for ss in subjs if ss not in no_sh]):
    baseline = epochs[s]['click'].get_data()
    el_resp = epochs[s]['el'].get_data(); sh_resp = epochs[s]['sh'].get_data()
    nepochs_click = baseline.shape[0]; nepochs_el = el_resp.shape[0]; nepochs_sh = sh_resp.shape[0]
    for ch_idx, ch in enumerate(ch_names[s]):
        el_ch_boots, sh_ch_boots = [], []
        for n in np.arange(nboots):
            # Get a random ten epochs and average across these
            baseline_idx = np.random.choice(np.arange(nepochs_click), size=10)
            sh_idx = np.random.choice(np.arange(nepochs_sh), size=10)
            el_idx = np.random.choice(np.arange(nepochs_el), size=10)
            # Average resp across samples/epochs (chunk of 10)
            baseline_chunk = baseline[baseline_idx,ch_idx,:].mean(0).mean(0)
            el_chunk = el_resp[el_idx,ch_idx,:].mean(0).mean(0)
            sh_chunk = sh_resp[sh_idx,ch_idx,:].mean(0).mean(0)
            if baseline_chunk > el_chunk:
                el_ch_boots.append(1)
            else:
                el_ch_boots.append(0)
            if baseline_chunk > sh_chunk:
                sh_ch_boots.append(1)
            else:
                sh_ch_boots.append(0)
        pvals['el'][s][ch] = sum(el_ch_boots)/nboots; pvals['sh'][s][ch] = sum(sh_ch_boots)/nboots

In [ ]:
# Save to dataframe
df = pd.DataFrame(columns = ['subj', 'ch_name', 'el_p', 'sh_p'])
for s in pvals['el'].keys():
    ch_names = list(pvals['el'][s])
    for ch in ch_names:
        new_row = pd.DataFrame({'subj':[s], 'ch_name':[ch],
                                'el_p':[pvals['el'][s][ch]], 'sh_p':[pvals['sh'][s][ch]]})
        df = df.append(new_row, ignore_index=True)
df.to_csv(os.path.join(git_path,"stats","bootstraps","csv",
    f"seeg_elec_significance_{len(pvals['el'].keys())}_subjs_{nboots}_boots_elsh.csv"), index=False)  